# Topic Modelling
Topic modelling (konu modelleme), büyük veri setlerinde metinler üzerinde çalışırken metinlerin konusunu tespit etmemizi sağlayan bir yaklaşımdır. Bu yaklaşımda, en yaygını *Latent Dirichlet Allocation (LDA) olmak üzere, NMF (Non-Negative Matrix Factorization), LSA (Latent Semantic Analysis), PLSA (Probabilistic latent Semantic Analysis)* gibi teknikler kullanılır.

### Kullanılacak verilerin elde edilmesi;

In [109]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
from bs4 import BeautifulSoup
import requests
import string
from wordcloud import STOPWORDS

#NLTK Libraries
import nltk
nltk.download('words')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing.preprocessing import remove_stopwords


# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

ModuleNotFoundError: No module named 'pandas.DataFrame'

In [42]:
def data_extraction(urls):
    texts = []
    for link in urls:
        txt = ''
        response = requests.get(link)
        content = response.content
        soup = BeautifulSoup(response.content, 'html.parser')
        for t in soup.find_all('p'):
            txt += t.get_text()
        texts.append(txt)
    return texts

def clear_text(texts):
    documents = []

    lemmatizer = WordNetLemmatizer()
    
    for item in range(0, len(texts)):
        
        document = texts[item].encode("ascii", errors="ignore").decode()
        
        # Remove all the special characters
        document = re.sub(r'\W', ' ', document)
    
        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
        
        # Remove single characters from the start
        document = re.sub(r'\b[a-zA-Z]\b', ' ', document)
        #document = re.sub(r'(?:^| )\w(?:$| )', ' ', document)
        
        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)
        
        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)
        
        # Removing numbers
        document = re.sub(r'[0-9]', '', document)
        
        # Converting to Lowercase
        document = document.lower()
        
        # Lemmatization
        document = document.split()
    
        document = [lemmatizer.lemmatize(word) for word in document]
        document = ' '.join(document)
        tokenized_doc = word_tokenize(document)
        tokens_without_sw = [word for word in tokenized_doc if word not in STOPWORDS]
        words = set(nltk.corpus.words.words())
        final_doc = [word for word in tokens_without_sw if word in words]
        
        documents.append(tokens_without_sw)
        
    return documents

def build_btgram(documents):

    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(documents, min_count=1, threshold=2, delimiter = b'_') # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[documents], min_count=1, threshold=2, delimiter = b'_')  
    
    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    
    def bigrams(texts):
        return [bigram_mod[doc] for doc in texts]
    
    def trigrams(texts):
        return [trigram_mod[bigram_mod[doc]] for doc in texts]
    
    item = bigrams(documents)
    item = trigrams(documents)
    for sent in [tokens_]:
        ttokens_ = trigram_phraser[sent]
    

    return documents

**Ben bu çalışmada, National Geographic websitesi üzerinden alınmış beş farklı konudaki makaleler üzerinde çalıştım. Bu makaleler;**

**- Storm Surges**

**- Green Sea Turtle**

**- Climate Change & Wildfires**

**- Plastic Pollution**

**- Supermoon**

**konularından oluşuyor. Dolayısıyla bu üç yöntemle elde edeceğimiz konuların bu şekilde olmasını bekliyoruz.**

In [191]:
urls = ['https://www.nationalgeographic.com/environment/article/storm-surges',
        'https://kids.nationalgeographic.com/animals/reptiles/facts/green-sea-turtle',
        'https://www.nationalgeographic.com/science/article/climate-change-increases-risk-fires-western-us',
        'https://www.nationalgeographic.com/science/article/plastic-pollution-huge-problem-not-too-late-to-fix-it',
        'https://kids.nationalgeographic.com/space/article/what-is-a-supermoon']

In [55]:
document = data_extraction(urls)
document = clear_text(document)


# LDA (Latent Dirichlet Allocation)
Topic modelling uygulamasında en yaygın kullanılan yöntemdir. Her belgenin farklı konuların koleksiyonundan ve her bir konunun farklı kelimelerin koleksiyonundan meydana geldiği fikrini temel alan, en basit kabul edilen bir konu modelleme örneğidir. LDA algoritması kullanıldığında sonuç olarak konuları belirleyen kelimeleri ve yüzdelerini elde ederiz. Python üzerinde uygulaması şu şekilde;

Döküman üzerinden bir sözlük ve o sözlük ile bir corpus oluşturuyoruz;

In [58]:
# Create Dictionary
id2word = corpora.Dictionary(document)
# Create Corpus
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in document]

Sonrasında gensim kütüphanesini kullanarak bir LDA modeli oluşturuyoruz. Bu modeli oluştururken kullandığımız *num_topics* parametresi kaç tane konu belirleneceğini ayarladığımız parametredir. **Coherence(tutarlılık) score** değerini en yüksek olarak alabildiğimiz topic sayısını önceden kontrol ederek bu değeri bu data seti için 50 olarak atadım. **Perplexity** değeri ise yine modelin ne kadar başarılı olduğunu gösteren bir parametredir. Bu değer ne kadar düşük olursa model o kadar iyi demektir.

In [51]:
from pprint import pprint
# number of topics
num_topics = 50
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=document, dictionary=id2word)
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

[(6,
  '0.029*"plastic" + 0.013*"waste" + 0.007*"million" + 0.007*"say" + '
  '0.007*"new" + 0.006*"production" + 0.006*"year" + 0.005*"climate" + '
  '0.005*"change" + 0.005*"fire"'),
 (29,
  '0.031*"plastic" + 0.011*"say" + 0.010*"waste" + 0.009*"new" + 0.008*"fire" '
  '+ 0.007*"production" + 0.007*"change" + 0.007*"climate" + 0.005*"ha" + '
  '0.005*"ton"'),
 (9,
  '0.014*"fire" + 0.006*"say" + 0.006*"ha" + 0.006*"forest" + 0.006*"climate" '
  '+ 0.005*"change" + 0.005*"heat" + 0.004*"plastic" + 0.004*"year" + '
  '0.004*"risk"'),
 (38,
  '0.020*"surge" + 0.018*"storm" + 0.011*"hurricane" + 0.009*"water" + '
  '0.009*"plastic" + 0.009*"ocean" + 0.006*"wind" + 0.005*"national" + '
  '0.004*"damage" + 0.004*"make"'),
 (18,
  '0.024*"plastic" + 0.012*"moon" + 0.010*"waste" + 0.009*"time" + '
  '0.009*"supermoon" + 0.007*"earth" + 0.007*"see" + 0.006*"percent" + '
  '0.006*"new" + 0.006*"million"'),
 (42,
  '0.024*"sea" + 0.019*"turtle" + 0.014*"green" + 0.008*"egg" + '
  '0.005*"nesti

Topic-keyword dağılımını daha net görebilmek için kaynak olarak yararlandığım internet sitesinde önerilen pyLDAvis ile gösterim yaptım. Burada büyük dairelere baktığımızda konu dağılımını daha net görebiliriz.

In [52]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19    -0.074847  0.005264       1        1  33.048363
35     0.069958  0.123999       2        1  32.343525
5      0.127075 -0.172802       3        1  11.020700
33    -0.150147 -0.042817       4        1   8.467735
1      0.057326  0.042769       5        1   5.608334
43    -0.133224 -0.035798       6        1   5.543580
37     0.103256 -0.111393       7        1   2.932682
32    -0.013470 -0.086246       8        1   0.329190
47     0.090520  0.138462       9        1   0.312771
49    -0.138983 -0.028324      10        1   0.250079
41     0.059648 -0.032005      11        1   0.003576
40     0.001237  0.038910      12        1   0.003576
39    -0.065070 -0.001732      13        1   0.003576
27     0.014355  0.016981      14        1   0.003576
42    -0.063622 -0.021888      15        1   0.003576
36    -0.057532 -0.017779      16        1   0.003576
44     0.045032 -0.092166      17        1   0.003576
34     0.025605  0.040895      18        1   0.003576
45     0.036316 -0.039540      19        1   0.003576
46    -0.016650  0.000067      20        1   0.003576
48    -0.051106 -0.007645      21        1   0.003576
31     0.028343 -0.070591      22        1   0.003576
30    -0.016648  0.034266      23        1   0.003576
29     0.012835  0.063605      24        1   0.003576
28     0.099092 -0.060843      25        1   0.003576
38     0.060108 -0.035047      26        1   0.003576
0      0.081569 -0.051108      27        1   0.003576
26    -0.029500  0.041379      28        1   0.003576
12     0.004141 -0.005461      29        1   0.003576
2      0.037394  0.058172      30        1   0.003576
3      0.025130  0.055999      31        1   0.003576
4     -0.027854  0.025923      32        1   0.003576
6      0.019186  0.064770      33        1   0.003576
7      0.007354  0.048045      34        1   0.003576
8      0.000806  0.027051      35        1   0.003576
9     -0.028955 -0.000779      36        1   0.003576
10    -0.049107 -0.015820      37        1   0.003576
11    -0.003672  0.037268      38        1   0.003576
13    -0.026580 -0.007749      39        1   0.003576
24    -0.058496 -0.018451      40        1   0.003576
14     0.101679 -0.010939      41        1   0.003576
15     0.004279  0.028865      42        1   0.003576
16    -0.093784 -0.020227      43        1   0.003576
17     0.028833  0.063483      44        1   0.003576
18     0.036380  0.061437      45        1   0.003576
20    -0.008520  0.023138      46        1   0.003576
21    -0.032188 -0.014911      47        1   0.003576
22    -0.020438  0.028542      48        1   0.003576
23    -0.016955 -0.079124      49        1   0.003576
25    -0.000110  0.011892      50        1   0.003576, topic_info=         Term       Freq      Total Category  logprob  loglift
1111  plastic  50.000000  50.000000  Default  30.0000  30.0000
488      fire  29.000000  29.000000  Default  29.0000  29.0000
1256    waste  18.000000  18.000000  Default  28.0000  28.0000
678       say  18.000000  18.000000  Default  27.0000  27.0000
193     storm  17.000000  17.000000  Default  26.0000  26.0000
...       ...        ...        ...      ...      ...      ...
1226      ton   0.000252   9.618487  Topic50  -5.9964  -0.3117
222     water   0.000316  18.812733  Topic50  -5.7696  -0.7558
230      will   0.000286  13.990868  Topic50  -5.8703  -0.5603
622   percent   0.000253  10.687330  Topic50  -5.9914  -0.4121
494    forest   0.000232   8.263632  Topic50  -6.0788  -0.2423

[3231 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
790       2  0.870651      ability
0         3  0.849938     abnormal
792       2  0.495719          acc
353       1  0.533703   accelerate
354       1  0.496817  accelerated
...     ...       ...          ...
350       2  0.386450         year
350       4  0.064408         year
350  

Burada ise her bir metnin konusunu oluşturan ana keyword'leri net bir şekilde gösteren tabloyu oluşturdum. 

In [105]:
def format_topics_sentences(model=lda_model, corpus=corpus, texts=document):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(model[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = model.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(model=lda_model, corpus=corpus, texts=document)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Topic Keywords', 'Text']

# Show
df_dominant_topic.head(5)


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Topic Keywords,Text
0,0,5.0,0.8026,"surge, storm, hurricane, water, wind, ocean, w...","[wind, make, hurricane, threat, storm, create,..."
1,1,33.0,0.9956,"sea, turtle, green, egg, nesting, will, water,...","[green, sea, turtle, world, largest, specie, h..."
2,2,19.0,0.9096,"fire, climate, change, ha, say, risk, heat, in...","[heating, planet, ha, driven, huge, increase, ..."
3,3,35.0,0.8082,"plastic, waste, new, production, million, will...","[correcting, plastic, waste, problem, requires..."
4,4,43.0,0.8676,"moon, fire, earth, time, supermoon, change, cl...","[find, make, moon, appear, extra, big, bright,..."


**Yukarıdaki tabloda, elde ettiğimiz topic keyword'lerin, üzerinde çalıştığımız makalelerle örtüştüğünü ve bu yöntemde başarılı bir sonuç elde ettiğimizi söyleyebiliriz.**

# LSA (Latent Semantic Analysis)


Aynı zamanda LSI(Latent Semantic Index) olarak da bilinir. LSA'nın amacı, sınıflandırma için boyutu azaltmaktır. Bu yöntemdeki temel fikir, sahip olduğumuz document-term matrisini document-topic matrisi ve topic-term matrisi olarak ikiye parçalamaktır. Ancak bunu yaparken her bir kelimenin belgelerdeki bulunma sayısı(raw count) yerine cosine similarity konusunda kullandığımız **tf-idf** değeri kullanılır. LSA genel olarak bir dimension reduction veya noise reducing tekniği olarak kullanılır. Bunu yaparken ise kelimeler ve belgeler arasındaki ilişkileri yakalayan topic keyword'leri bulabilmek için SVD(singular value decomposition) yöntemini kullanır. Bu bileşenler, gensim kütüphanesinin sağladığı LsiModel içerisinde bulunur.

Anlaması ve uygulaması en kolay metoddur. Diğer algoritmalara göre daha hızlıdır çünkü yalnızca decomposition işlemi uygular. Ancak bunun yanında kelimelerin farklı anlamlarını tespit etme konusunda başarısızdır. LDA'ya göre daha zordur ve daha az doğru sonuçlar verir.

Uygulamasını yaptığımızda ise, topic keyword'lerinin yine üzerinde çalıştığımız dökümanlarla örtüştüğünü görüyoruz.

In [104]:
from gensim.models import LsiModel
document = data_extraction(urls)
document = clear_text(document)

# Create Dictionary
id2word = corpora.Dictionary(document)
# Create Corpus
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in document]

# generate LSA model
lsamodel = LsiModel(corpus, num_topics=15, id2word = id2word)  # train model
pprint(lsamodel.print_topics(num_topics=15, num_words=10))

# Compute Coherence Score
coherence_model_lsa = CoherenceModel(model=lsamodel, texts=document, dictionary=id2word)
coherence_lsa = coherence_model_lsa.get_coherence()
print('\nCoherence Score: ', coherence_lsa)

[(0,
  '0.607*"plastic" + 0.243*"waste" + 0.175*"say" + 0.174*"new" + '
  '0.139*"million" + 0.134*"fire" + 0.121*"production" + 0.115*"will" + '
  '0.114*"year" + 0.110*"ton"'),
 (1,
  '0.430*"fire" + -0.325*"plastic" + 0.211*"climate" + 0.158*"heat" + '
  '0.152*"risk" + 0.151*"ha" + 0.143*"forest" + 0.143*"change" + '
  '-0.130*"waste" + 0.116*"across"'),
 (2,
  '-0.565*"storm" + -0.475*"surge" + -0.316*"hurricane" + -0.234*"water" + '
  '-0.158*"wind" + -0.131*"ocean" + -0.113*"inland" + -0.090*"wall" + '
  '-0.090*"create" + -0.088*"damage"'),
 (3,
  '0.584*"sea" + 0.523*"turtle" + 0.370*"green" + 0.155*"egg" + '
  '0.123*"nesting" + 0.092*"shell" + 0.081*"will" + 0.063*"distance" + '
  '0.062*"color" + 0.062*"flipper"'),
 (4,
  '-0.528*"moon" + -0.301*"earth" + -0.301*"supermoon" + -0.296*"time" + '
  '-0.151*"brighter" + -0.151*"full" + -0.151*"appear" + -0.149*"look" + '
  '-0.148*"larger" + -0.147*"month"')]

Coherence Score:  0.5560375140435166


# NMF (Non-Negative Matrix Factorization)

Yüksek boyutlu vektörlerin, çarpanlarına ayrılarak düşük boyutlu vektörler haline getirilmesini temel alan yöntemdir. Ancak adından da anlaşılacağı üzere içerdiği vektörlerin negatif olmaması gerekmektedir. Elimizdeki ana matrisi(X) kullanarak, X = WH olacak şekilde üç matris elde etmiş oluyoruz. Bu matrisler;

**- (X)Document-Word Matrix: Hangi belgelerde hangi kelimelerin göründüğü bilgisini içerir**

**- (W)Basis Vectors: Belgeler üzerinden elde edilen konu kümeleri**

**- (H)Coefficient Matrix: Her bir belgedeki konuların ağırlıklarının bulunduğu matris**

şeklindedir. Topic modelling konusunda bu yöntemi kullanırken ise her bir belge için kelimelere anlamlarına göre bir ağırlık atanmasını sağlıyoruz ve ağırlığı en yüksek kelimeler kümesinin, belgenin konusunu gösterdiğini düşünerek hareket ediyoruz. 

In [188]:
document = data_extraction(urls)
documents = clear_text(document)

# Create Dictionary
id2word = corpora.Dictionary(documents)
# Create Corpus
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in documents]

In [189]:
from sklearn.decomposition import NMF
# use tfidf by removing tokens that don't appear in at least 50 documents
vect = TfidfVectorizer(stop_words='english')
 
# Fit and transform
X = vect.fit_transform(document)
nmf = NMF(n_components=5, random_state=5)
nmf.fit(X)
W = nmf.transform(X)
H = nmf.components_

# Create a DataFrame: components_df
components_df = pd.DataFrame(H, columns=vect.get_feature_names())

D:\Anaconda\lib\site-packages\sklearn\decomposition\_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


In [190]:
for topic in range(components_df.shape[0]):
    tmp = components_df.iloc[topic]
    print(f'Topic {topic+1}:')
    print(tmp.nlargest(10))
    print('\n')

Topic 1:
plastic          0.897475
waste            0.438765
new              0.257450
plastics         0.239327
production       0.219383
tons             0.199439
million          0.176997
says             0.144815
billion          0.139607
environmental    0.139607
Name: 0, dtype: float64


Topic 2:
sea          0.625461
green        0.395028
turtles      0.345265
eggs         0.131676
nesting      0.131676
turtle       0.131676
adult        0.065838
color        0.065838
distances    0.065838
female       0.065838
Name: 1, dtype: float64


Topic 3:
storm         0.555103
surges        0.313754
surge         0.193079
hurricane     0.168945
hurricanes    0.168945
water         0.163166
winds         0.144810
inland        0.120675
ocean         0.116831
create        0.096540
Name: 2, dtype: float64


Topic 4:
moon         0.571886
earth        0.326792
supermoon    0.326792
appear       0.163396
brighter     0.163396
14           0.131827
looks        0.131827
time         0.116789


## Kaynaklar;

-https://www.nationalgeographic.com/environment/article/storm-surges

-https://kids.nationalgeographic.com/animals/reptiles/facts/green-sea-turtle

-https://www.nationalgeographic.com/science/article/climate-change-increases-risk-fires-western-us

-https://www.nationalgeographic.com/science/article/plastic-pollution-huge-problem-not-too-late-to-fix-it

-https://kids.nationalgeographic.com/space/article/what-is-a-supermoon




-https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

-https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

-https://medium.com/@busragokmen67/latent-dirichlet-allocation-lda-kullanarak-nas%C4%B1l-topic-modeling-yap%C4%B1l%C4%B1r-75fe8dddcdd2



-https://medium.com/nanonets/topic-modeling-with-lsa-psla-lda-and-lda2vec-555ff65b0b05

-https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

-https://forestforthetree.com/statistics/2018/01/28/topic-modelling-with-lsa-and-lda.html

-https://towardsdatascience.com/2-latent-methods-for-dimension-reduction-and-topic-modeling-20ff6d7d547



-https://medium.com/ml2vec/topic-modeling-is-an-unsupervised-learning-approach-to-clustering-documents-to-discover-topics-fdfbf30e27df

-https://medium.com/voice-tech-podcast/topic-modelling-using-nmf-2f510d962b6e

-https://predictivehacks.com/topic-modelling-with-nmf-in-python/

-https://towardsdatascience.com/topic-modeling-articles-with-nmf-8c6b2a227a45
